In [45]:
%cd ~/Research/goingBALD
%load_ext autoreload
%autoreload 2

/home/minh/Research/goingBALD
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import os
from bald.data import conll2003_utils as data_utils

In [47]:
DATA_TRAIN_PATH = "artifacts/data/raw/CoNLL2003/eng.train"

In [48]:
DATA_PROCESSED_DIR = "artifacts/data/processed/CoNLL2003/"

In [49]:
sentences = data_utils.load_raw_dataset(DATA_TRAIN_PATH)

In [52]:
# generate tags
tags = data_utils.generate_NER_tag_set(sentences)

In [53]:
tags.save(os.path.join(DATA_PROCESSED_DIR, "tags_idx.txt"))

In [54]:
tags.idx2key

['O', 'PER', 'ORG', 'LOC', 'MISC']

# generating word2vec on data

In [55]:
from bald.parameters import WORD_EMBEDDING_SIZE

In [56]:
from gensim.models.word2vec import Word2Vec

In [36]:
from nltk.tokenize import word_tokenize

In [57]:
# processing coNLL data to list of raw sentences, 1 per line
# TODO do we need to lowercase the words and strip symbols
RAW_SENTENCE_FNAME =  os.path.join(DATA_PROCESSED_DIR, "eng.train.raw_sentences.txt")
raw_sentences = []
word_set = set()
with open(RAW_SENTENCE_FNAME, 'w') as f:
    for sentence in sentences:
        orig_sentence = [word['word'] for word in sentence]
        raw_sentences.append(orig_sentence)
        f.write(" ".join(raw_sentence) + "\n")

In [72]:
# using CBOW
# negative is negative sampling (https://towardsdatascience.com/nlp-101-negative-sampling-and-glove-936c88f3bc68)
model = Word2Vec(raw_sentences, sg=1, size=WORD_EMBEDDING_SIZE, negative=8, window=5, min_count=1)

In [73]:
word_vectors = model.wv

In [74]:
word_vectors.vectors.shape

(23623, 300)

In [75]:
word_vectors.save_word2vec_format(
    os.path.join(DATA_PROCESSED_DIR, "word2vec.txt"),
    fvocab=os.path.join(DATA_PROCESSED_DIR, "word2vec_vocab.txt"))

In [76]:
import numpy as np
np.save(os.path.join(DATA_PROCESSED_DIR, "word2vec.vector.npy"), word_vectors.vectors)

In [77]:
from gensim.models.keyedvectors import KeyedVectors

In [78]:
wv = KeyedVectors.load_word2vec_format(os.path.join(DATA_PROCESSED_DIR, "word2vec.txt"))

In [79]:
word_vectors.vectors.shape

(23623, 300)

In [83]:
model.similarity("hardened", "throughput")

<ipython-input-83-92d3f34f90f8>:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  model.similarity("hardened", "throughput")


0.9985663

In [99]:
# we want the word indxer for word2vec
from bald.data.indexer import Vocabulary
word2vec_vocab = Vocabulary()
for word in word_vectors.index2word:
    word2vec_vocab.add(word)
word2vec_vocab.save(os.path.join(DATA_PROCESSED_DIR, "word2vec_vocab_idx.txt"))

In [108]:
word2vec_vocab.idx2key[:10]

['<pad>', '<unk>', '.', ',', 'the', 'of', 'in', 'to', 'a', '(']

# generating dataset

In [120]:
from bald.data.indexer import Vocabulary, Indexer
tag_set = Indexer()
word2vec_vocab = Vocabulary()
tag_set.load(os.path.join(DATA_PROCESSED_DIR, "tags_idx.txt"))
word2vec_vocab.load(os.path.join(DATA_PROCESSED_DIR, "word2vec_vocab_idx.txt"))

raw_sentences = data_utils.load_raw_dataset(DATA_TRAIN_PATH)

In [121]:
#training data
sentences = []
tags = []
for sentence in raw_sentences:
    sentence_idx = []
    tag_idx = []
    for word_data in sentence:
        sentence_idx.append(word2vec_vocab[word_data['word']])
        tag_idx.append(tag_set[word_data['NER_tag']])
    sentences.append(sentence_idx)
    tags.append(tag_idx)

In [127]:
import pickle
pickle.dump(sentences, open(os.path.join(DATA_PROCESSED_DIR, "sentences.train.pkl"), 'wb'))
pickle.dump(tags, open(os.path.join(DATA_PROCESSED_DIR, "NER_tags.train.pkl"), 'wb'))

In [128]:
y